In [1]:
import pandas as pd

# ANN models
import torch
import ML_Models.ANN.model as model_ann

# (Train & Test) Loaders
import ML_Models.data_loader as loaders

# Explanation Models
from explainers.lime import LIME
from explainers.shap_explainer import SHAPExplainer
from explainers.grad import Gradient
from explainers.ebp import EBP
from explainers.integrated_gradients import IntegratedGradients
from explainers.guided_backprop import GuidedBackprop
from explainers.smoothgrad import SmoothGrad
from explainers.gradcam import GradCAM
from explainers.guided_gradcam import GuidedGradCAM
from explainers.lrp import LRP
from explainers.input_x_gradient import InputTimesGradient

# Perturbation Methods
import torch.distributions as tdist
from explainers.perturbation_methods import RandomPerturbation
from explainers.perturbation_methods import UniformPerturbation
from explainers.perturbation_methods import BootstrapPerturbation
from explainers.perturbation_methods import MarginalPerturbation
from explainers.perturbation_methods import AdversarialPerturbation

# Import Evaluation Methods
from evaluator import Evaluator

# lime utils
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json

# torch utils
import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
import os

In [3]:
'''
Loading Data Loaders
'''

# this has to be fixed for the moment as the ANN model is trained on this data

params = {
    'n_samples': 1000,
    'dim': 20,
    'n_clusters': 10,
    'distance_to_center': 6,
    'test_size': 0.25,
    'upper_weight': 1,
    'lower_weight': -1,
    'seed': 564,
    'sigma': None,
    'sparsity': 0.25
}

#loader_train_cifar, loader_test_mnist = loaders.return_loaders(data_name='cifar10', is_tabular=False, batch_size=1)
loader_train_gauss, loader_test_gauss = loaders.return_loaders(data_name='gaussian', is_tabular=True,
                                                               batch_size=5, gauss_params=params)

In [4]:
import ML_Models.data_loader as loaders
#loader_tr_cifar, loader_te_mnist = loaders.return_loaders(data_name='cifar10', is_tabular=False, batch_size=1)
loader_tr_gauss, loader_te_gauss = loaders.return_loaders(data_name='gaussian', is_tabular=True,
                                                               batch_size=5, gauss_params=params)

In [5]:
'''
Gaussian Data DGP
'''
gauss_train_input = loader_train_gauss.dataset.ground_truth_dict
data_iter = iter(loader_train_gauss)
inputs, labels, weights, masks, masked_weights, probs, cluster_idx = data_iter.next()
gaussian_all = torch.FloatTensor(loader_train_gauss.dataset.data)

In [6]:
'''
Load ML Models
'''
model_path = 'ML_Models/Saved_Models/ANN/gaussian_lr_0.002_acc_0.91.pt'
ann = model_ann.ANN_softmax(20, hidden_layer_1=100, num_of_classes=2)
ann.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Testing whether the outputs work
a = ann(inputs.float())
b = ann.predict(inputs.float())
c = ann.predict_proba(inputs.float())

In [7]:
inp = inputs.detach().float()[0,:].reshape(1,-1)
lab = labels.type(torch.int64)[0]

In [8]:
# this is my sample of interest [1, 20], therefore we apply reshape(-1) when we use the perturbation methods below
inp

tensor([[0.3687, 0.2564, 0.2482, 0.2223, 0.3330, 0.6314, 0.2301, 0.3010, 0.2758,
         0.2113, 0.4511, 0.5057, 0.6350, 0.6913, 0.6565, 0.5608, 0.1720, 0.1148,
         0.3965, 0.3110]])

In [9]:
# those will be my data samples for the bootstrapped method below
data_samples = inputs.detach().float()

In [10]:
# size of the data samples for the bootstrap perturbations
data_samples.shape

torch.Size([5, 20])

In [11]:
# show actual samples
data_samples

tensor([[0.3687, 0.2564, 0.2482, 0.2223, 0.3330, 0.6314, 0.2301, 0.3010, 0.2758,
         0.2113, 0.4511, 0.5057, 0.6350, 0.6913, 0.6565, 0.5608, 0.1720, 0.1148,
         0.3965, 0.3110],
        [0.7040, 0.3417, 0.4224, 0.2571, 0.3039, 0.2761, 0.3564, 0.2528, 0.2377,
         0.1400, 0.4421, 0.3737, 0.5200, 0.3879, 0.5097, 0.6795, 0.3858, 0.4028,
         0.3159, 0.6617],
        [0.3995, 0.3047, 0.2831, 0.3034, 0.3003, 0.3001, 0.3407, 0.4103, 0.7625,
         0.3852, 0.5477, 0.3525, 0.6206, 0.5275, 0.6039, 0.5971, 0.4006, 0.3004,
         0.4707, 0.6525],
        [0.3390, 0.2415, 0.2577, 0.3175, 0.3584, 0.3761, 0.8605, 0.3469, 0.2316,
         0.1793, 0.2995, 0.6684, 0.6059, 0.5728, 0.4847, 0.5867, 0.4978, 0.6036,
         0.5176, 0.7156],
        [0.2748, 0.1610, 0.3119, 0.2756, 0.2680, 0.4208, 0.2265, 0.7890, 0.3026,
         0.2550, 0.2556, 0.6277, 0.2430, 0.6615, 0.3701, 0.6505, 0.4216, 0.5838,
         0.5706, 0.6603]])

# Testing Perturbation Methods on Tabular Data

In [12]:
'''
Testing Perturbation Methods
'''

feature_mask = masks[0]
inp = inputs.detach().float()[0,:].reshape(1,-1)
data_samples = inputs.detach().float()

print("Generating Random Perturbations....")
perturbation_method = RandomPerturbation("tabular")
num_samples = 10
max_distance = 4
perturbed_inp = perturbation_method.get_perturbed_inputs(inp.reshape(-1), feature_mask, num_samples, max_distance)
print(perturbed_inp.shape)

print("Generating Uniform Perturbations....")
perturbation_method = UniformPerturbation("tabular")
perturbed_inp = perturbation_method.get_perturbed_inputs(inp.reshape(-1), feature_mask, num_samples, max_distance)
print(perturbed_inp.shape)

print("Generating Marginal Perturbations....")
perturbation_method = MarginalPerturbation("tabular", 
                                           [tdist.Normal(0,5), tdist.Normal(1,6), 
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6), 
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6),
                                            tdist.Normal(0,5), tdist.Normal(1,6)])
num_samples = 100
max_distance = 30
perturbed_inp_m = perturbation_method.get_perturbed_inputs(inp.reshape(-1), feature_mask, num_samples, max_distance)
print(perturbed_inp_m.shape)

print("Generating Bootstrap Perturbations....")
perturbation_method = BootstrapPerturbation("tabular")
num_samples = 3
max_distance = 100
perturbed_inp = perturbation_method.get_perturbed_inputs(inp.reshape(-1), feature_mask, num_samples, max_distance, data_samples)
print(perturbed_inp.shape)

Generating Random Perturbations....
torch.Size([10, 20])
Generating Uniform Perturbations....
torch.Size([10, 20])
Generating Marginal Perturbations....
torch.Size([43, 20])
Generating Bootstrap Perturbations....
torch.Size([3, 20])


To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# Testing Explanations Methods on Tabular Data

In [13]:
'''
Testing SHAP on Gaussian Data
Note that this is Explaining P(y|x)
'''
shap = SHAPExplainer(ann.predict, gaussian_all[0:100,:].detach())
exp_shap = shap.get_explanation(inp).numpy()
exp_shap = pd.DataFrame(exp_shap)
print('shape:', exp_shap.shape)
exp_shap.head()


shape: (1, 20)


l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.066147,0.0,0.038725,0.0,0.006073,0.042875,-0.031176,-0.010332,-0.006598,0.007894,0.0,-0.002524,-0.087667,0.0,0.0,0.0,0.036734,0.107897,-0.075051,-0.108214


In [14]:
'''
Testing LIME on Gaussian Data
Note that this is Explaining P(y|x)
'''
lime = LIME(ann.predict, gaussian_all)
exp_lime = lime.get_explanation(inp).numpy()
exp_lime = pd.DataFrame(exp_lime)
print('shape:', exp_lime.shape)
exp_lime.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.68s/it]

shape: (1, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.040293,-0.005993,0.040429,-0.006859,0.039962,0.018967,-0.024141,-0.038742,-0.033103,0.044965,0.001036,-0.004944,-0.069791,0.003876,0.004197,0.009208,0.020861,0.008258,-0.097125,-0.088714


In [15]:
'''
Testing Grad on Gaussian Data
Note that this is Explaining f(x), i.e., the logit score
All methods below explain the logit score
'''

grad = Gradient(ann)
exp_grad = grad.get_explanation(inp, lab)
#exp_grad = pd.DataFrame(exp_grad)
#exp_grad.head()

Input Tensor 0 did not already require gradients, required_grads has been set automatically.


In [16]:
'''
Testing IG on Gaussian Data
'''
ig = IntegratedGradients(ann)
exp_ig = ig.get_explanation(inp, lab).numpy()
exp_ig = pd.DataFrame(exp_ig)
exp_ig.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.144879,-0.056206,0.047392,-0.046201,0.018836,0.68644,-0.06687,0.027381,0.100618,-0.281676,0.017795,-0.013474,-0.681632,-0.043585,-0.083226,0.015318,-0.047155,-0.039344,0.10547,0.448124


In [17]:
'''
Testing InputTimesGradient on Gaussian Data
'''
itg = InputTimesGradient(ann)
exp_itg = itg.get_explanation(inp, lab).detach().numpy()
exp_itg = pd.DataFrame(exp_itg)
exp_itg.head()

Input Tensor 0 did not already require gradients, required_grads has been set automatically.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.391584,0.097225,0.067636,-0.105617,-0.454665,2.697986,-0.36067,0.048904,0.17998,-1.240863,0.067635,-0.008622,-3.296016,-0.145805,-0.156001,0.14531,-0.216947,-0.178625,0.188929,1.662914


In [18]:
'''
Testing SmoothGrad on Gaussian Data
'''
sg = SmoothGrad(ann)
exp_sg = sg.get_explanation(inp, lab).numpy()
exp_sg = pd.DataFrame(exp_sg)
exp_sg.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.07917,0.031182,0.035213,0.041974,0.066472,0.022435,0.05428,0.094949,0.082361,0.1051,0.005519,0.011075,0.085278,0.010248,0.058124,0.007741,0.024389,0.015419,0.145726,0.060176


In [19]:
'''
Testing EBP on Gaussian Data
'''
ebp = EBP(ann)
exp_ebp = ebp.get_explanation(inp, lab)
#exp_ebp = pd.DataFrame(exp_ebp)
#exp_ebp.head()

In [20]:
'''
Testing GradCAM on Gaussian Data
'''
#gradcam = GradCAM(ann, ann.input2)
#exp_gradcam = gradcam.get_explanation(inp, lab)
#exp_gradcam = pd.DataFrame(exp_gradcam)
#exp_gradcam.head()

'\nTesting GradCAM on Gaussian Data\n'

In [21]:
'''
Testing GuidedGradCAM on Gaussian Data
'''
#ggc = GuidedGradCAM(ann, ann.input1)
#ggc_exp = ggc.get_explanation(inp, lab)
#ggc_exp = pd.DataFrame(ggc_exp)
#ggc_exp.head()

'\nTesting GuidedGradCAM on Gaussian Data\n'

In [22]:
'''
Testing LRP on Gaussian Data
'''
#lrp = LRP(ann)
#exp_lrp = lrp.get_explanation(inp, lab)
#exp_lrp = pd.DataFrame(exp_lrp)
#exp_lrp.head()

'\nTesting LRP on Gaussian Data\n'

In [23]:
inputs[0].unsqueeze(0).shape

torch.Size([1, 20])

In [24]:
input_dict = {}
input_dict['x'] = inputs[0].reshape(-1)
input_dict['explainer'] = grad
input_dict['explanation_x'] = exp_grad
perturbation = UniformPerturbation("tabular")
input_dict['perturbation'] = perturbation
input_dict['L_map'] = ann.L_relu
input_dict['p_norm'] = 2
input_dict['top_k'] = 1
input_dict['eval_metric'] = 'eval_relative_stability'
input_dict['perturb_max_distance'] = 4
input_dict['perturb_method'] = 'RandomPerturbation'
input_dict['y'] = labels[0].detach().item()
input_dict['y_pred'] = torch.max(ann(inputs[0].unsqueeze(0).float()), 1).indices.detach().item()

In [25]:
'explanation_x' in input_dict

True

In [26]:
evaluator = Evaluator(input_dict)
stability, stability_ratios, rep_diffs, x_diffs, exp_diffs = evaluator.eval_relative_stability(delta=1e10)

{'x': tensor([0.3687, 0.2564, 0.2482, 0.2223, 0.3330, 0.6314, 0.2301, 0.3010, 0.2758,
        0.2113, 0.4511, 0.5057, 0.6350, 0.6913, 0.6565, 0.5608, 0.1720, 0.1148,
        0.3965, 0.3110], dtype=torch.float64), 'explainer': Gradient(
  (model): ANN_softmax(
    (input1): Linear(in_features=20, out_features=100, bias=True)
    (input2): Linear(in_features=100, out_features=2, bias=True)
    (relu): ReLU()
    (softmax): Softmax(dim=1)
  )
), 'explanation_x': tensor([[1.0621, 0.3792, 0.2725, 0.4752, 1.3652, 4.2729, 1.5677, 0.1625, 0.6526,
         5.8718, 0.1499, 0.0170, 5.1906, 0.2109, 0.2376, 0.2591, 1.2610, 1.5559,
         0.4765, 5.3472]]), 'perturbation': <explainers.perturbation_methods.UniformPerturbation object at 0x0000026D3A25C2E8>, 'L_map': <bound method ANN_softmax.L_relu of ANN_softmax(
  (input1): Linear(in_features=20, out_features=100, bias=True)
  (input2): Linear(in_features=100, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)>, 'p_norm': 2,

AttributeError: 'str' object has no attribute 'get_perturbed_inputs'